In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import packets and libraries
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation, BatchNormalization, LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2,ResNet50

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras import backend as K

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
IMAGE_SIZE = 256
EPOCHS = 30
BATCH = 10
LR = 1e-4

PATH = "/content/drive/MyDrive//OTU-2D-Dataset-main/OTU-2D-Dataset-main/dataset_split"

In [ ]:
import os
from glob import glob
import cv2
import numpy as np
import tensorflow as tf

def load_data(path):
    def load_split(split):
        if split == 'train':
            image_dir = os.path.join(path, split, 'augmented_speckle')
            original_images = sorted(glob(os.path.join(image_dir, 'original_*.JPG')))
            haze_images = sorted(glob(os.path.join(image_dir, 'speckle_*.JPG')))
            images = original_images + haze_images

            masks = sorted(glob(os.path.join(path, split, 'masks', '*')))
            if len(masks) * 2 != len(images):
                raise ValueError(f"Số lượng mask x2 ({len(masks)*2}) không bằng số lượng ảnh ({len(images)})")

            masks = masks * 2
        else:
            images = sorted(glob(os.path.join(path, split, 'images', '*')))
            masks = sorted(glob(os.path.join(path, split, 'masks', '*')))

        paired = list(zip(images, masks))
        np.random.shuffle(paired)
        images, masks = zip(*paired)
        return list(images), list(masks)

    train_x, train_y = load_split('train')
    valid_x, valid_y = load_split('validation')
    test_x, test_y   = load_split('test')
    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


In [ ]:
def read_image(path):
    if isinstance(path, bytes):
        path = path.decode('utf-8')
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    if x is None:
        raise ValueError(f"Could not read image: {path}")
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = x.astype(np.float32) / 255.0
    return x

def read_mask(path):
    if isinstance(path, bytes):
        path = path.decode('utf-8')
    y = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if y is None:
        raise ValueError(f"Could not read mask: {path}")
    y = cv2.resize(y, (IMAGE_SIZE, IMAGE_SIZE))
    y = y.astype(np.float32) / 255.0
    _, y = cv2.threshold(y, 0, 1, cv2.THRESH_BINARY)
    y = np.expand_dims(y, axis=-1)
    return y

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    y.set_shape([IMAGE_SIZE, IMAGE_SIZE, 1])
    return x, y

def tf_dataset(x, y, batch=8, repeat=True):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=len(x))
    dataset = dataset.batch(batch)
    if repeat:
        dataset = dataset.repeat()
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [ ]:
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(PATH)

print("Training data:", len(train_x))
print("Validation data:", len(valid_x))
print("Testing data:", len(test_x))

In [ ]:
train_dataset = tf_dataset(train_x, train_y, batch=BATCH, repeat=True)
valid_dataset = tf_dataset(valid_x, valid_y, batch=BATCH, repeat=True)
test_dataset  = tf_dataset(test_x, test_y, batch=BATCH, repeat=False)


In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy

# Constants
beta = 0.25
alpha = 0.26
gamma = 2.3
epsilon = 1e-7
smooth = 1e-15

# Dice Coefficient
def dice_coef(y_true, y_pred):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + epsilon) / (
            tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + epsilon)

# Sensitivity
def sensitivity(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1)))
    possible_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true, 0, 1)))
    return true_positives / (possible_positives + epsilon)

# Specificity
def specificity(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_negatives = tf.reduce_sum(tf.round(tf.clip_by_value((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = tf.reduce_sum(tf.round(tf.clip_by_value(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + epsilon)

# Convert probabilities to logits
def convert_to_logits(y_pred):
    y_pred = tf.clip_by_value(y_pred, K.epsilon(), 1 - K.epsilon())
    return tf.math.log(y_pred / (1 - y_pred))

# Weighted cross entropy
def weighted_cross_entropyloss(y_true, y_pred):
    y_pred = convert_to_logits(y_pred)
    pos_weight = beta / (1 - beta)
    loss = tf.nn.weighted_cross_entropy_with_logits(logits=y_pred, labels=y_true, pos_weight=pos_weight)
    return tf.reduce_mean(loss)

# Dice Loss
def generalized_dice_coefficient(y_true, y_pred):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - generalized_dice_coefficient(y_true, y_pred)

# Confusion metrics
def confusion(y_true, y_pred):
    y_pred = tf.cast(tf.clip_by_value(y_pred, 0, 1), tf.float32)
    y_true = tf.cast(tf.clip_by_value(y_true, 0, 1), tf.float32)
    y_pred_neg = 1 - y_pred
    y_neg = 1 - y_true
    tp = tf.reduce_sum(y_true * y_pred)
    fp = tf.reduce_sum(y_neg * y_pred)
    fn = tf.reduce_sum(y_true * y_pred_neg)
    precision = (tp + smooth) / (tp + fp + smooth)
    recall = (tp + smooth) / (tp + fn + smooth)
    return precision, recall

# True positive
def true_positive(y_true, y_pred):
    y_pred = tf.round(tf.clip_by_value(y_pred, 0, 1))
    y_true = tf.round(tf.clip_by_value(y_true, 0, 1))
    return (tf.reduce_sum(y_true * y_pred) + smooth) / (tf.reduce_sum(y_true) + smooth)

# True negative
def true_negative(y_true, y_pred):
    y_pred = tf.round(tf.clip_by_value(y_pred, 0, 1))
    y_true = tf.round(tf.clip_by_value(y_true, 0, 1))
    y_pred_neg = 1 - y_pred
    y_neg = 1 - y_true
    return (tf.reduce_sum(y_neg * y_pred_neg) + smooth) / (tf.reduce_sum(y_neg) + smooth)

# Jaccard
def jaccard_similarity(y_true, y_pred):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f + y_pred_f - y_true_f * y_pred_f)
    return intersection / (union + epsilon)

def jaccard_loss(y_true, y_pred):
    return 1 - jaccard_similarity(y_true, y_pred)

# Binary Dice
def binary_dice(y_true, y_pred):
    return 0.5 * binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)



In [ ]:
import tensorflow as tf
import os
import random
import numpy as np
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from keras.utils import normalize
from tqdm import tqdm

from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers as L
from tensorflow.keras.models import Model

# Convolutional block
def conv_block(x, num_filters):
    x = L.Conv2D(num_filters, 3, padding="same", kernel_initializer='he_normal', activation="relu")(x)
    x = L.Conv2D(num_filters, 3, padding="same", kernel_initializer='he_normal', activation="relu")(x)
    return x

# Encoder block
def encoder_block(x, num_filters):
    x = conv_block(x, num_filters)
    p = L.MaxPool2D((2, 2))(x)
    return x, p

# Decoder block
def decoder_block(x, s, num_filters):
    x = L.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(x)
    x = L.Concatenate()([x, s])
    x = conv_block(x, num_filters)
    return x

# Build U-Net model
def build_unet(input_shape):
    inputs = L.Input(input_shape)

    c1, p1 = encoder_block(inputs, 64)
    c2, p2 = encoder_block(p1, 128)
    c3, p3 = encoder_block(p2, 256)
    c4, p4 = encoder_block(p3, 512)

    c5 = conv_block(p4, 1024)

    d6 = decoder_block(c5, c4, 512)
    d7 = decoder_block(d6, c3, 256)
    d8 = decoder_block(d7, c2, 128)
    d9 = decoder_block(d8, c1, 64)

    outputs = L.Conv2D(1, 1, padding="same", activation="sigmoid")(d9)

    model = Model(inputs, outputs)
    return model


In [ ]:
input = (256, 256, 3)
model = build_unet(input)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 256, 256, 64)   │          1,792 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 256, 256, 64)   │         36,928 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 128, 128, 64)   │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 128, 128, 128)  │         73,856 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 128, 128, 128)  │        147,584 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 64, 64, 128)    │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 64, 64, 256)    │        295,168 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 64, 64, 256)    │        590,080 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 32, 32, 256)    │              0 │ conv2d_5[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 32, 32, 512)    │      1,180,160 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 32, 32, 512)    │      2,359,808 │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3           │ (None, 16, 16, 512)    │              0 │ conv2d_7[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 16, 16, 1024)   │      4,719,616 │ max_pooling2d_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, 16, 16, 1024)   │      9,438,208 │ conv2d_8[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose          │ (None, 32, 32, 512)    │      2,097,664 │ conv2d_9[0][0]         │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 32, 32, 1024)   │              0 │ conv2d_transpose[0][0… │
│                      

 Total params: 31,031,745 (118.38 MB)

 Trainable params: 31,031,745 (118.38 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_path = "unet_with_and_without_speckle.h5"

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False),
    checkpoint_callback
]

In [ ]:
train_steps = len(train_x) // BATCH
valid_steps = len(valid_x) // BATCH

if len(train_x) % BATCH != 0:
    train_steps += 1
if len(valid_x) % BATCH != 0:
    valid_steps += 1

print(train_steps)
print(valid_steps)

236
15


In [ ]:

opt = tf.keras.optimizers.Nadam(LR,clipnorm=1.0)
metrics = [dice_coef, jaccard_similarity, tf.keras.metrics.Recall(), tf.keras.metrics.Precision()]

model.compile(loss=dice_loss, optimizer=opt, metrics=metrics)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=30,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=callbacks
)


Epoch 1/30
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 703ms/step - dice_coef: 0.4690 - jaccard_similarity: 0.3271 - loss: 0.5310 - precision: 0.3186 - recall: 0.7174
Epoch 1: val_loss improved from inf to 0.29869, saving model to unet_with_and_without_speckle.h5


236/236 ━━━━━━━━━━━━━━━━━━━━ 589s 1s/step - dice_coef: 0.4695 - jaccard_similarity: 0.3276 - loss: 0.5305 - precision: 0.3192 - recall: 0.7174 - val_dice_coef: 0.7037 - val_jaccard_similarity: 0.5498 - val_loss: 0.2987 - val_precision: 0.6510 - val_recall: 0.7866 - learning_rate: 1.0000e-04
Epoch 2/30
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 513ms/step - dice_coef: 0.7035 - jaccard_similarity: 0.5491 - loss: 0.2965 - precision: 0.6793 - recall: 0.7516
Epoch 2: val_loss did not improve from 0.29869
236/236 ━━━━━━━━━━━━━━━━━━━━ 134s 569ms/step - dice_coef: 0.7035 - jaccard_similarity: 0.5491 - loss: 0.2964 - precision: 0.6794 - recall: 0.7517 - val_dice_coef: 0.6792 - val_jaccard_similarity: 0.5289 - val_loss: 0.3239 - val_precision: 0.8338 - val_recall: 0.6065 - learning_rate: 1.0000e-04
Epoch 3/30
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 517ms/step - dice_coef: 0.7441 - jaccard_similarity: 0.5986 - loss: 0.2559 - precision: 0.7536 - recall: 0.7571
Epoch 3: val_loss did not improve from 0.29869
236/236 ━

236/236 ━━━━━━━━━━━━━━━━━━━━ 157s 551ms/step - dice_coef: 0.7746 - jaccard_similarity: 0.6385 - loss: 0.2254 - precision: 0.7783 - recall: 0.7956 - val_dice_coef: 0.7817 - val_jaccard_similarity: 0.6464 - val_loss: 0.2193 - val_precision: 0.8656 - val_recall: 0.7178 - learning_rate: 1.0000e-04
Epoch 5/30
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 514ms/step - dice_coef: 0.7876 - jaccard_similarity: 0.6549 - loss: 0.2124 - precision: 0.7953 - recall: 0.7955
Epoch 5: val_loss did not improve from 0.21932
236/236 ━━━━━━━━━━━━━━━━━━━━ 198s 739ms/step - dice_coef: 0.7877 - jaccard_similarity: 0.6550 - loss: 0.2123 - precision: 0.7953 - recall: 0.7955 - val_dice_coef: 0.7773 - val_jaccard_similarity: 0.6396 - val_loss: 0.2231 - val_precision: 0.7129 - val_recall: 0.8776 - learning_rate: 1.0000e-04
Epoch 6/30
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 515ms/step - dice_coef: 0.8147 - jaccard_similarity: 0.6919 - loss: 0.1853 - precision: 0.8123 - recall: 0.8299
Epoch 6: val_loss improved from 0.21932 to 0.18620, s

236/236 ━━━━━━━━━━━━━━━━━━━━ 131s 554ms/step - dice_coef: 0.8147 - jaccard_similarity: 0.6919 - loss: 0.1853 - precision: 0.8124 - recall: 0.8299 - val_dice_coef: 0.8142 - val_jaccard_similarity: 0.6895 - val_loss: 0.1862 - val_precision: 0.8635 - val_recall: 0.7810 - learning_rate: 1.0000e-04
Epoch 7/30
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 512ms/step - dice_coef: 0.8427 - jaccard_similarity: 0.7317 - loss: 0.1573 - precision: 0.8480 - recall: 0.8479
Epoch 7: val_loss did not improve from 0.18620
236/236 ━━━━━━━━━━━━━━━━━━━━ 198s 749ms/step - dice_coef: 0.8427 - jaccard_similarity: 0.7317 - loss: 0.1573 - precision: 0.8480 - recall: 0.8479 - val_dice_coef: 0.8102 - val_jaccard_similarity: 0.6840 - val_loss: 0.1898 - val_precision: 0.8658 - val_recall: 0.7641 - learning_rate: 1.0000e-04
Epoch 8/30
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 513ms/step - dice_coef: 0.8501 - jaccard_similarity: 0.7436 - loss: 0.1499 - precision: 0.8528 - recall: 0.8604
Epoch 8: val_loss improved from 0.18620 to 0.17864, s

236/236 ━━━━━━━━━━━━━━━━━━━━ 146s 618ms/step - dice_coef: 0.8501 - jaccard_similarity: 0.7437 - loss: 0.1499 - precision: 0.8528 - recall: 0.8605 - val_dice_coef: 0.8176 - val_jaccard_similarity: 0.6972 - val_loss: 0.1786 - val_precision: 0.9051 - val_recall: 0.7559 - learning_rate: 1.0000e-04
Epoch 9/30
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 514ms/step - dice_coef: 0.8780 - jaccard_similarity: 0.7853 - loss: 0.1220 - precision: 0.8875 - recall: 0.8772
Epoch 9: val_loss improved from 0.17864 to 0.17141, saving model to unet_with_and_without_speckle.h5


...

236/236 ━━━━━━━━━━━━━━━━━━━━ 144s 609ms/step - dice_coef: 0.9068 - jaccard_similarity: 0.8309 - loss: 0.0932 - precision: 0.9144 - recall: 0.9057 - val_dice_coef: 0.8395 - val_jaccard_similarity: 0.7254 - val_loss: 0.1600 - val_precision: 0.8467 - val_recall: 0.8449 - learning_rate: 1.0000e-04
Epoch 12/30
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 515ms/step - dice_coef: 0.9097 - jaccard_similarity: 0.8365 - loss: 0.0903 - precision: 0.9186 - recall: 0.9103
Epoch 12: val_loss did not improve from 0.16001
236/236 ━━━━━━━━━━━━━━━━━━━━ 128s 542ms/step - dice_coef: 0.9097 - jaccard_similarity: 0.8366 - loss: 0.0903 - precision: 0.9186 - recall: 0.9103 - val_dice_coef: 0.8323 - val_jaccard_similarity: 0.7147 - val_loss: 0.1702 - val_precision: 0.8934 - val_recall: 0.7799 - learning_rate: 1.0000e-04
Epoch 13/30
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 513ms/step - dice_coef: 0.9280 - jaccard_similarity: 0.8666 - loss: 0.0720 - precision: 0.9375 - recall: 0.9216
Epoch 13: val_loss improved from 0.16001 to 0.1545

236/236 ━━━━━━━━━━━━━━━━━━━━ 142s 545ms/step - dice_coef: 0.9280 - jaccard_similarity: 0.8666 - loss: 0.0720 - precision: 0.9375 - recall: 0.9216 - val_dice_coef: 0.8449 - val_jaccard_similarity: 0.7330 - val_loss: 0.1546 - val_precision: 0.8826 - val_recall: 0.8186 - learning_rate: 1.0000e-04
Epoch 14/30
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 511ms/step - dice_coef: 0.9346 - jaccard_similarity: 0.8782 - loss: 0.0654 - precision: 0.9414 - recall: 0.9318
Epoch 14: val_loss did not improve from 0.15456
236/236 ━━━━━━━━━━━━━━━━━━━━ 127s 538ms/step - dice_coef: 0.9346 - jaccard_similarity: 0.8782 - loss: 0.0654 - precision: 0.9414 - recall: 0.9318 - val_dice_coef: 0.8321 - val_jaccard_similarity: 0.7184 - val_loss: 0.1673 - val_precision: 0.8812 - val_recall: 0.8139 - learning_rate: 1.0000e-04
Epoch 15/30
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 512ms/step - dice_coef: 0.9425 - jaccard_similarity: 0.8921 - loss: 0.0575 - precision: 0.9514 - recall: 0.9390
Epoch 15: val_loss improved from 0.15456 to 0.1536

236/236 ━━━━━━━━━━━━━━━━━━━━ 143s 606ms/step - dice_coef: 0.9425 - jaccard_similarity: 0.8921 - loss: 0.0575 - precision: 0.9514 - recall: 0.9390 - val_dice_coef: 0.8465 - val_jaccard_similarity: 0.7364 - val_loss: 0.1536 - val_precision: 0.9017 - val_recall: 0.8088 - learning_rate: 1.0000e-04
Epoch 16/30
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 513ms/step - dice_coef: 0.9506 - jaccard_similarity: 0.9063 - loss: 0.0494 - precision: 0.9581 - recall: 0.9451
Epoch 16: val_loss did not improve from 0.15365
236/236 ━━━━━━━━━━━━━━━━━━━━ 126s 533ms/step - dice_coef: 0.9506 - jaccard_similarity: 0.9062 - loss: 0.0494 - precision: 0.9581 - recall: 0.9451 - val_dice_coef: 0.8410 - val_jaccard_similarity: 0.7314 - val_loss: 0.1602 - val_precision: 0.8507 - val_recall: 0.8509 - learning_rate: 1.0000e-04
Epoch 17/30
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 510ms/step - dice_coef: 0.9515 - jaccard_similarity: 0.9079 - loss: 0.0485 - precision: 0.9584 - recall: 0.9471
Epoch 17: val_loss did not improve from 0.15365
23

236/236 ━━━━━━━━━━━━━━━━━━━━ 128s 540ms/step - dice_coef: 0.9535 - jaccard_similarity: 0.9115 - loss: 0.0465 - precision: 0.9601 - recall: 0.9497 - val_dice_coef: 0.8563 - val_jaccard_similarity: 0.7509 - val_loss: 0.1446 - val_precision: 0.8828 - val_recall: 0.8351 - learning_rate: 1.0000e-04
Epoch 19/30
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 512ms/step - dice_coef: 0.9623 - jaccard_similarity: 0.9276 - loss: 0.0377 - precision: 0.9670 - recall: 0.9600
Epoch 19: val_loss did not improve from 0.14463
236/236 ━━━━━━━━━━━━━━━━━━━━ 139s 591ms/step - dice_coef: 0.9622 - jaccard_similarity: 0.9275 - loss: 0.0378 - precision: 0.9670 - recall: 0.9600 - val_dice_coef: 0.8119 - val_jaccard_similarity: 0.6893 - val_loss: 0.1879 - val_precision: 0.8921 - val_recall: 0.7616 - learning_rate: 1.0000e-04
Epoch 20/30
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 512ms/step - dice_coef: 0.9554 - jaccard_similarity: 0.9151 - loss: 0.0446 - precision: 0.9618 - recall: 0.9522
Epoch 20: val_loss did not improve from 0.14463
23

236/236 ━━━━━━━━━━━━━━━━━━━━ 148s 628ms/step - dice_coef: 0.9662 - jaccard_similarity: 0.9349 - loss: 0.0338 - precision: 0.9706 - recall: 0.9645 - val_dice_coef: 0.8572 - val_jaccard_similarity: 0.7524 - val_loss: 0.1436 - val_precision: 0.8766 - val_recall: 0.8414 - learning_rate: 1.0000e-04
Epoch 22/30
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 509ms/step - dice_coef: 0.9719 - jaccard_similarity: 0.9454 - loss: 0.0281 - precision: 0.9755 - recall: 0.9695
Epoch 22: val_loss did not improve from 0.14363
236/236 ━━━━━━━━━━━━━━━━━━━━ 136s 575ms/step - dice_coef: 0.9718 - jaccard_similarity: 0.9454 - loss: 0.0282 - precision: 0.9755 - recall: 0.9695 - val_dice_coef: 0.8510 - val_jaccard_similarity: 0.7467 - val_loss: 0.1505 - val_precision: 0.8865 - val_recall: 0.8238 - learning_rate: 1.0000e-04
Epoch 23/30
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 510ms/step - dice_coef: 0.9724 - jaccard_similarity: 0.9467 - loss: 0.0276 - precision: 0.9766 - recall: 0.9715
Epoch 23: val_loss did not improve from 0.14363
23

In [ ]:
model.save_weights("/content/drive/MyDrive/unet_with_and_without_speckle_weights.weights.h5")


In [ ]:
test_dataset = tf_dataset(test_x, test_y, batch=BATCH)

test_steps = (len(test_x)//BATCH)
if len(test_x) % BATCH != 0:
    test_steps += 1
model.evaluate(test_dataset, steps=test_steps)

15/15 ━━━━━━━━━━━━━━━━━━━━ 44s 186ms/step - dice_coef: 0.8259 - jaccard_similarity: 0.7096 - loss: 0.1742 - precision: 0.8480 - recall: 0.8146


[0.15536577999591827,
 0.8454007506370544,
 0.7389343976974487,
 0.8272062540054321,
 0.8747991323471069]

In [ ]:
import numpy as np
import cv2
import os
from tqdm import tqdm

def predict_and_save(model, image_path, mask_path, output_dir):
    # Đọc ảnh
    img = read_image(image_path.encode('utf-8'))
    true_mask = read_mask(mask_path.encode('utf-8'))

    img_input = np.expand_dims(img, axis=0)
    pred_mask = model.predict(img_input)[0]


    name = os.path.basename(image_path).split('.')[0]
    out_path = os.path.join(output_dir, name)
    os.makedirs(out_path, exist_ok=True)

    # Lưu ảnh gốc
    img_save = (img * 255).astype(np.uint8)
    img_save = cv2.cvtColor(img_save, cv2.COLOR_RGB2BGR)
    cv2.imwrite(os.path.join(out_path, f"{name}_image.jpg"), img_save)

    # Lưu mask thật
    mask_save = (true_mask * 255).astype(np.uint8)
    cv2.imwrite(os.path.join(out_path, f"{name}_true_mask.png"), mask_save)

    # Lưu mask dự đoán
    pred_mask_save = (pred_mask * 255).astype(np.uint8)
    cv2.imwrite(os.path.join(out_path, f"{name}_pred_mask.png"), pred_mask_save)

def process_predictions(model, data, output_dir):
    x_data, y_data = data
    os.makedirs(output_dir, exist_ok=True)

    for x_path, y_path in tqdm(zip(x_data, y_data), total=len(x_data)):
        predict_and_save(model, x_path, y_path, output_dir)


In [ ]:
output_dir = "/content/drive/MyDrive//OTU-2D-Dataset-main/OTU-2D-Dataset-main/dataset_split_/prediction_with_and_without_speckle"
process_predictions(model, (test_x, test_y), output_dir)


  0%|          | 0/146 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


  1%|          | 1/146 [00:00<00:24,  6.04it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


  1%|▏         | 2/146 [00:00<00:22,  6.33it/s]

...

 99%|█████████▊| 144/146 [00:24<00:00,  6.11it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step


 99%|█████████▉| 145/146 [00:25<00:00,  6.32it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


100%|██████████| 146/146 [00:25<00:00,  5.77it/s]
